In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
#import the library
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import BatchNorm1d
from torch.nn.utils import clip_grad_norm_
from torch.optim.lr_scheduler import StepLR
import numpy as np
import torch
from sklearn.metrics import f1_score


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
#imoort the dataset
training_dataset = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
testing_dataset = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [4]:
#check the dataset
training_dataset.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Read the dataset, acknowledge what each column means.
target:If it's about the real disaster, it's a 1. If not, it's a 0.

In [5]:
testing_dataset.head()
#It doesn't have the target, which we need to predict.

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
#Use the train_test_split to split the data
#test_size = 0.2,We make 20% of the dataset to be the validation dataset
trn_data,val_data,trn_labels,val_labels = train_test_split(training_dataset['text'],training_dataset['target'],test_size = 0.2,random_state=10)

In [7]:
#Now we get the text data
trn_data.head()

2572    @MsMigot wow what convincing &amp; compelling ...
1813    http://t.co/iGXRqPoTm7 Bin Laden family plane ...
2767    70 Years After Atomic Bombs Japan Still Strugg...
6248    @PyrBliss ah I remember those days. In a snows...
7563                            I wrecked my stomach help
Name: text, dtype: object

In [8]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
def preprocess_text(text, tokenizer, max_length):
    #将文本转换成BERT可以理解的格式
    encoding = tokenizer.encode_plus(
        text,
        #处理后文本的最大长度。如果文本超过这个长度，它会被截断。
        max_length=max_length,
        #如果文本超过max_length，这个选项会使文本被截断。
        truncation=True,
        #如果文本短于max_length，这个选项将以填充的方式扩展文本长度至max_length。
        padding='max_length',
        #return_tensors='pt': 指定返回的数据类型为PyTorch张量（tensor）。
        return_tensors='pt'
    )
    
    return {
        'input_ids': encoding['input_ids'].squeeze(),
        'attention_mask': encoding['attention_mask'].squeeze()
    }

In [10]:
max_length = 128
train_encodings = trn_data.apply(lambda x: preprocess_text(x, tokenizer, max_length))
val_encodings = val_data.apply(lambda x: preprocess_text(x, tokenizer, max_length))
test_encodings = testing_dataset['text'].apply(lambda x: preprocess_text(x, tokenizer, max_length))

In [11]:
train_input_ids = torch.stack([enc['input_ids'] for enc in train_encodings])
train_attention_mask = torch.stack([enc['attention_mask'] for enc in train_encodings])
train_labels = torch.tensor(trn_labels.values)

val_input_ids = torch.stack([enc['input_ids'] for enc in val_encodings])
val_attention_mask = torch.stack([enc['attention_mask'] for enc in val_encodings])
val_labels = torch.tensor(val_labels.values)

test_input_ids = torch.stack([enc['input_ids'] for enc in test_encodings])
test_attention_mask = torch.stack([enc['attention_mask'] for enc in test_encodings])

In [12]:
train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_mask)

In [13]:
optimizer = AdamW([
    {'params': model.roberta.parameters(), 'lr': 2e-5},  # 修改这里
    {'params': model.classifier.parameters(), 'lr': 1e-3}
], lr=5e-6)
criterion = torch.nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=1, gamma=0.25)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [15]:
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [16]:
torch.cuda.empty_cache()

In [17]:
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        # 梯度裁剪
        clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

    # 学习率调度程序更新
    scheduler.step()

    model.eval()
    val_loss = 0.0
    correct_preds = 0
    all_preds = []  # 初始化列表以收集所有预测
    all_labels = []  # 初始化列表以收集所有标签

    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            correct_preds += torch.sum(preds == labels).item()
            all_preds.extend(preds.cpu().numpy())  # 收集预测
            all_labels.extend(labels.cpu().numpy())  # 收集标签

    val_accuracy = correct_preds / len(val_data)
    f1 = f1_score(all_labels, all_preds, average='macro')  # 计算F1分数
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, F1 Score: {f1:.4f}")

Epoch 1/5, Validation Loss: 9.9634, Validation Accuracy: 0.8293, F1 Score: 0.8265
Epoch 2/5, Validation Loss: 9.2785, Validation Accuracy: 0.8319, F1 Score: 0.8255
Epoch 3/5, Validation Loss: 9.6500, Validation Accuracy: 0.8326, F1 Score: 0.8281
Epoch 4/5, Validation Loss: 9.8091, Validation Accuracy: 0.8332, F1 Score: 0.8289
Epoch 5/5, Validation Loss: 9.7076, Validation Accuracy: 0.8339, F1 Score: 0.8294


In [18]:
model.eval()
predictions = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask = batch  # 不再需要 labels
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        predictions.extend(preds.cpu().numpy())

In [19]:
submission_df = pd.DataFrame({'id': testing_dataset['id'], 'target': predictions})
submission_df.to_csv('/kaggle/working/submission.csv', index=False)